In [2]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
df = pd.read_csv('data/heart_disease.csv')
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalach   303 non-null    int64  
 8   exang     303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    int64  
 11  ca        303 non-null    int64  
 12  thal      303 non-null    int64  
 13  target    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


In [4]:
# Split data into X and y
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

### XGB GridSearch

In [5]:
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold, GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import cross_val_score
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=2)

In [6]:
scores = cross_val_score(XGBClassifier(), X, y, cv=kfold)
print('Accuracy:', np.round(scores, 2))
print('Accuracy mean: %0.2f' % (scores.mean()))

[13:40:08] WARNING: ..\src\learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[13:40:08] WARNING: ..\src\learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[13:40:08] WARNING: ..\src\learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[13:40:08] WARNING: ..\src\learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

In [7]:
def grid_search_xg(params, random=False): 
    xgb = XGBClassifier(random_state=2)
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=2)
    if random:
        grid = RandomizedSearchCV(xgb, params, cv=kfold, n_iter=50, n_jobs=-1, random_state=2)
    else:
        grid = GridSearchCV(xgb, params, cv=kfold, n_jobs=-1)
    
    grid.fit(X, y)

    best_params = grid.best_params_
    print("Best params:", best_params)
    best_score = grid.best_score_
    print("Best score: {:.5f}".format(best_score))

In [8]:
grid_search_xg(params={'n_estimators':[50,100, 200, 300]})

[13:40:12] WARNING: ..\src\learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Best params: {'n_estimators': 50}
Best score: 0.78907


In [9]:
grid_search_xg(params={'learning_rate':[0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5]})

[13:40:13] WARNING: ..\src\learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Best params: {'learning_rate': 0.05}
Best score: 0.79585


In [10]:
grid_search_xg(params={'max_depth':[2, 3, 5, 6, 8]})

[13:40:14] WARNING: ..\src\learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Best params: {'max_depth': 2}
Best score: 0.79902


In [11]:
grid_search_xg(params={'gamma':[0, 0.01, 0.1, 0.5, 1, 2]})  # gamma=minimum sample size

[13:40:14] WARNING: ..\src\learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Best params: {'gamma': 0.5}
Best score: 0.79574


In [12]:
grid_search_xg(params={'min_child_weight':[1, 2, 3, 4, 5]})

[13:40:15] WARNING: ..\src\learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Best params: {'min_child_weight': 5}
Best score: 0.81219


In [13]:
grid_search_xg(params={'subsample':[0.5, 0.7, 0.8, 0.9, 1]})

[13:40:16] WARNING: ..\src\learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Best params: {'subsample': 0.8}
Best score: 0.79579


In [14]:
grid_search_xg(params={'colsample_bytree':[0.5, 0.7, 0.8, 0.9, 1]})

[13:40:16] WARNING: ..\src\learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Best params: {'colsample_bytree': 0.9}
Best score: 0.79885


In [15]:
grid_search_xg(params={'n_estimators':[30,50,100],'learning_rate':[0.03,0.05,0.07],
                      'max_depth':[1,2,3],'gamma':[0.3,0.5,0.7], 'min_child_weight':[3,5,7],
                      'subsample':[0.75,0.8,0.85],'colsample_bytree':[0.85,0.9,0.95]},random=True)

[13:40:22] WARNING: ..\src\learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Best params: {'subsample': 0.8, 'n_estimators': 100, 'min_child_weight': 7, 'max_depth': 1, 'learning_rate': 0.07, 'gamma': 0.3, 'colsample_bytree': 0.9}
Best score: 0.83847


In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [17]:
# X_train과 Y_train을 다시 분할해서 사용 (train, validation으로 사용)
# X_val, y_val을 이용해서 하이퍼파라미터 조정
X_train_p, X_val, y_train_p, y_val = train_test_split(X_train, y_train, random_state=2)
model = XGBClassifier(learning_rate=0.07,
                      max_depth=1,
                      gamma=0.7,
                      colsample_bytree=0.95,
                      colsample_bynode=0.9,
                      colsample_bylevel=0.5,
                      min_child_weight=7,
                      subsample=0.75,
                      random_state=2, 
                      n_estimators=5000)
eval_set = [(X_val, y_val)]

# 10번까지 evaluation set에서 진전이 안보이면 early stopping 적용
model.fit(X_train_p, y_train_p, early_stopping_rounds=10, eval_set=eval_set)

[13:40:28] WARNING: ..\src\learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0]	validation_0-logloss:0.68369
[1]	validation_0-logloss:0.67625
[2]	validation_0-logloss:0.67012
[3]	validation_0-logloss:0.65246
[4]	validation_0-logloss:0.64994
[5]	validation_0-logloss:0.64520
[6]	validation_0-logloss:0.63026
[7]	validation_0-logloss:0.62672
[8]	validation_0-logloss:0.62561
[9]	validation_0-logloss:0.61697
[10]	validation_0-logloss:0.60454
[11]	validation_0-logloss:0.59419
[12]	validation_0-logloss:0.58354
[13]	validation_0-logloss:0.57549
[14]	validation_0-logloss:0.57368
[15]	validation_0-logloss:0.57096
[16]	validation_0-logloss:0.56985
[17]	validation_0-logloss:0.56830
[18]	validation_0-logloss:0.56491
[19]	validation_0-logloss:0.56414
[20]	validation_0-logloss:0.55813
[21]	validation_0-logloss:0.55785
[22]	valid

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=0.5,
              colsample_bynode=0.9, colsample_bytree=0.95, gamma=0.7, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.07, max_delta_step=0, max_depth=1,
              min_child_weight=7, missing=nan, monotone_constraints='()',
              n_estimators=5000, n_jobs=8, num_parallel_tree=1, random_state=2,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=0.75,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [18]:
from sklearn.metrics import accuracy_score
model_xg = XGBClassifier(n_estimators=97,  # 위에서 97번째에서 진전이 없음 (107번째까지 가보기)
                         learning_rate=0.07,
                         max_depth=1,
                         gamma=0.7,
                         colsample_bytree=0.95,
                         colsample_bynode=0.9,
                         colsample_bylevel=0.5,
                         min_child_weight=7,
                         subsample=0.75,
                         random_state=2)
model_xg.fit(X_train, y_train)  # X_train_p나 y_train_p를 사용하지 않음
# X_train, y_train을 사용함으로써 더 많은 데이터를 이용해서 fit (X_val, y_val는 하이퍼파라미터 조정용)
y_tpred=model_xg.predict(X_train)

accuracy_t=accuracy_score(y_train,y_tpred)
print('Train_accuracy',accuracy_t)

# predictions for test data
y_pred = model_xg.predict(X_test)

# evaluate predictions
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

[13:40:36] WARNING: ..\src\learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Train_accuracy 0.8722466960352423
Accuracy: 85.53%


### LGBM GridSearch

In [19]:
from lightgbm import LGBMClassifier

def grid_search_lgb(params, random=False): 
    
    lgb = LGBMClassifier(boosting_type='gbdt',random_state=2)
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=2)
    
    if random:
        grid = RandomizedSearchCV(lgb, params, cv=kfold, n_iter=50, n_jobs=-1, random_state=2)
    else:
        grid = GridSearchCV(lgb, params, cv=kfold, n_jobs=-1)
    
    grid.fit(X_train, y_train)

    best_params = grid.best_params_
    print("Best params:", best_params)
    best_score = grid.best_score_
    print("Best score: {:.5f}".format(best_score))

In [20]:
grid_search_lgb(params={'n_estimators':[50,100, 200, 300]})

Best params: {'n_estimators': 100}
Best score: 0.81527


In [21]:
grid_search_lgb(params={'learning_rate':[0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5]})

Best params: {'learning_rate': 0.05}
Best score: 0.81961


In [22]:
grid_search_lgb(params={'max_depth':[2, 3, 5, 6, 8]})

Best params: {'max_depth': 3}
Best score: 0.81546


In [23]:
grid_search_lgb(params={'min_child_weight':[1, 2, 3, 4, 5]})

Best params: {'min_child_weight': 2}
Best score: 0.82860


In [24]:
grid_search_lgb(params={'subsample':[0.5, 0.7, 0.8, 0.9, 1]})

Best params: {'subsample': 0.5}
Best score: 0.81527


In [25]:
grid_search_lgb(params={'colsample_bytree':[0.5, 0.7, 0.8, 0.9, 1]})

Best params: {'colsample_bytree': 0.8}
Best score: 0.83295


In [26]:
grid_search_lgb(params={'n_estimators':[50,100,150],'learning_rate':[0.05,0.1],
                      'max_depth':[2,3,4],'min_child_weight':[1,2,3],
                      'subsample':[0.4,0.5,0.6],'colsample_bytree':[0.75,0.80,0.85]},random=True)

Best params: {'subsample': 0.6, 'n_estimators': 50, 'min_child_weight': 2, 'max_depth': 4, 'learning_rate': 0.1, 'gamma': 0, 'colsample_bytree': 0.8}
Best score: 0.83275


In [27]:
model = LGBMClassifier(learning_rate=0.1,
                       max_depth=4,
                       colsample_bytree=0.8,
                       min_child_weight=2,
                       subsample=0.8, 
                       random_state=2, 
                       n_estimators=5000)
eval_set = [(X_val, y_val)]
model.fit(X_train_p, y_train_p, early_stopping_rounds=10, eval_set=eval_set)

[1]	valid_0's binary_logloss: 0.671593
Training until validation scores don't improve for 10 rounds
[2]	valid_0's binary_logloss: 0.647058
[3]	valid_0's binary_logloss: 0.618075
[4]	valid_0's binary_logloss: 0.604443
[5]	valid_0's binary_logloss: 0.594897
[6]	valid_0's binary_logloss: 0.592324
[7]	valid_0's binary_logloss: 0.588504
[8]	valid_0's binary_logloss: 0.588728
[9]	valid_0's binary_logloss: 0.57682
[10]	valid_0's binary_logloss: 0.553374
[11]	valid_0's binary_logloss: 0.539731
[12]	valid_0's binary_logloss: 0.526648
[13]	valid_0's binary_logloss: 0.514976
[14]	valid_0's binary_logloss: 0.509412
[15]	valid_0's binary_logloss: 0.500868
[16]	valid_0's binary_logloss: 0.49735
[17]	valid_0's binary_logloss: 0.488447
[18]	valid_0's binary_logloss: 0.489834
[19]	valid_0's binary_logloss: 0.478443
[20]	valid_0's binary_logloss: 0.477505
[21]	valid_0's binary_logloss: 0.473294
[22]	valid_0's binary_logloss: 0.473079
[23]	valid_0's binary_logloss: 0.478133
[24]	valid_0's binary_logloss:

LGBMClassifier(colsample_bytree=0.8, max_depth=4, min_child_weight=2,
               n_estimators=5000, random_state=2, subsample=0.8)

In [28]:
model_lgbc = LGBMClassifier(n_estimators=27,
                            max_depth=4,
                            colsample_bytree=0.8,
                            min_child_weight=2,
                            subsample=0.6, 
                            random_state=2)

model_lgbc.fit(X_train, y_train)
y_tpred=model_lgbc.predict(X_train)

t_accuracy=accuracy_score(y_train,y_tpred)
print('Training accuracy',t_accuracy)

# predictions for test data
y_pred = model_lgbc.predict(X_test)

# evaluate predictions
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Training accuracy 0.9030837004405287
Accuracy: 84.21%


### CatBoost GridSearch

In [68]:
from catboost import CatBoostClassifier

def grid_search_cat(params, random=False): 
    cat = CatBoostClassifier(random_state=2)
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=2)
    if random:
        grid = RandomizedSearchCV(cat, params, cv=kfold, n_iter=50, n_jobs=-1, random_state=2)
    else:
        grid = GridSearchCV(cat, params, cv=kfold, n_jobs=-1)
    
    grid.fit(X_train, y_train)

    best_params = grid.best_params_
    print("Best params:", best_params)
    best_score = grid.best_score_
    print("Best score: {:.5f}".format(best_score))

In [60]:
grid_search_cat(params={'n_estimators':[700,1000,1500]})

Learning rate set to 0.007586
0:	learn: 0.6874851	total: 147ms	remaining: 1m 42s
1:	learn: 0.6820752	total: 150ms	remaining: 52.4s
2:	learn: 0.6758678	total: 153ms	remaining: 35.6s
3:	learn: 0.6711769	total: 154ms	remaining: 26.8s
4:	learn: 0.6647500	total: 156ms	remaining: 21.7s
5:	learn: 0.6596311	total: 159ms	remaining: 18.3s
6:	learn: 0.6540520	total: 161ms	remaining: 16s
7:	learn: 0.6489188	total: 163ms	remaining: 14.1s
8:	learn: 0.6432248	total: 166ms	remaining: 12.7s
9:	learn: 0.6388055	total: 168ms	remaining: 11.6s
10:	learn: 0.6335689	total: 170ms	remaining: 10.6s
11:	learn: 0.6277067	total: 172ms	remaining: 9.87s
12:	learn: 0.6227897	total: 174ms	remaining: 9.21s
13:	learn: 0.6181008	total: 177ms	remaining: 8.69s
14:	learn: 0.6150229	total: 179ms	remaining: 8.16s
15:	learn: 0.6115261	total: 180ms	remaining: 7.69s
16:	learn: 0.6067535	total: 181ms	remaining: 7.28s
17:	learn: 0.6017691	total: 183ms	remaining: 6.94s
18:	learn: 0.5971248	total: 185ms	remaining: 6.63s
19:	learn: 0

172:	learn: 0.2893145	total: 485ms	remaining: 1.48s
173:	learn: 0.2883972	total: 487ms	remaining: 1.47s
174:	learn: 0.2877718	total: 489ms	remaining: 1.47s
175:	learn: 0.2870705	total: 491ms	remaining: 1.46s
176:	learn: 0.2863323	total: 493ms	remaining: 1.46s
177:	learn: 0.2853161	total: 495ms	remaining: 1.45s
178:	learn: 0.2842390	total: 496ms	remaining: 1.44s
179:	learn: 0.2834980	total: 498ms	remaining: 1.44s
180:	learn: 0.2827166	total: 500ms	remaining: 1.43s
181:	learn: 0.2819735	total: 502ms	remaining: 1.43s
182:	learn: 0.2814736	total: 503ms	remaining: 1.42s
183:	learn: 0.2807201	total: 504ms	remaining: 1.41s
184:	learn: 0.2798056	total: 506ms	remaining: 1.41s
185:	learn: 0.2786783	total: 508ms	remaining: 1.4s
186:	learn: 0.2778759	total: 509ms	remaining: 1.4s
187:	learn: 0.2770589	total: 511ms	remaining: 1.39s
188:	learn: 0.2764150	total: 513ms	remaining: 1.39s
189:	learn: 0.2755170	total: 515ms	remaining: 1.38s
190:	learn: 0.2745431	total: 516ms	remaining: 1.38s
191:	learn: 0.

406:	learn: 0.1686270	total: 954ms	remaining: 687ms
407:	learn: 0.1683015	total: 955ms	remaining: 684ms
408:	learn: 0.1679930	total: 957ms	remaining: 681ms
409:	learn: 0.1675685	total: 959ms	remaining: 679ms
410:	learn: 0.1674035	total: 961ms	remaining: 676ms
411:	learn: 0.1670082	total: 963ms	remaining: 673ms
412:	learn: 0.1669493	total: 965ms	remaining: 670ms
413:	learn: 0.1667580	total: 967ms	remaining: 668ms
414:	learn: 0.1664545	total: 969ms	remaining: 665ms
415:	learn: 0.1661947	total: 971ms	remaining: 663ms
416:	learn: 0.1659145	total: 973ms	remaining: 660ms
417:	learn: 0.1656286	total: 975ms	remaining: 658ms
418:	learn: 0.1654727	total: 977ms	remaining: 655ms
419:	learn: 0.1652854	total: 979ms	remaining: 653ms
420:	learn: 0.1648999	total: 981ms	remaining: 650ms
421:	learn: 0.1647203	total: 984ms	remaining: 648ms
422:	learn: 0.1646097	total: 985ms	remaining: 645ms
423:	learn: 0.1641864	total: 987ms	remaining: 642ms
424:	learn: 0.1636775	total: 989ms	remaining: 640ms
425:	learn: 

635:	learn: 0.1142600	total: 1.43s	remaining: 144ms
636:	learn: 0.1140398	total: 1.43s	remaining: 141ms
637:	learn: 0.1139146	total: 1.43s	remaining: 139ms
638:	learn: 0.1137627	total: 1.43s	remaining: 137ms
639:	learn: 0.1135720	total: 1.43s	remaining: 134ms
640:	learn: 0.1134964	total: 1.44s	remaining: 132ms
641:	learn: 0.1132983	total: 1.44s	remaining: 130ms
642:	learn: 0.1130830	total: 1.44s	remaining: 128ms
643:	learn: 0.1129241	total: 1.44s	remaining: 125ms
644:	learn: 0.1127840	total: 1.44s	remaining: 123ms
645:	learn: 0.1125707	total: 1.44s	remaining: 121ms
646:	learn: 0.1122613	total: 1.45s	remaining: 118ms
647:	learn: 0.1119588	total: 1.45s	remaining: 116ms
648:	learn: 0.1118291	total: 1.45s	remaining: 114ms
649:	learn: 0.1115504	total: 1.45s	remaining: 112ms
650:	learn: 0.1113004	total: 1.45s	remaining: 109ms
651:	learn: 0.1112308	total: 1.46s	remaining: 107ms
652:	learn: 0.1111135	total: 1.46s	remaining: 105ms
653:	learn: 0.1109773	total: 1.46s	remaining: 103ms
654:	learn: 

In [61]:
grid_search_cat(params={'learning_rate':[0.01, 0.03, 0.05,0.1]})

0:	learn: 0.6856956	total: 2.77ms	remaining: 2.77s
1:	learn: 0.6786074	total: 5.16ms	remaining: 2.57s
2:	learn: 0.6705083	total: 7.1ms	remaining: 2.36s
3:	learn: 0.6644318	total: 8.24ms	remaining: 2.05s
4:	learn: 0.6561096	total: 10.4ms	remaining: 2.07s
5:	learn: 0.6495173	total: 12.7ms	remaining: 2.11s
6:	learn: 0.6423642	total: 15.1ms	remaining: 2.15s
7:	learn: 0.6358267	total: 17.3ms	remaining: 2.15s
8:	learn: 0.6285760	total: 19.8ms	remaining: 2.18s
9:	learn: 0.6225406	total: 22.4ms	remaining: 2.21s
10:	learn: 0.6159768	total: 24.9ms	remaining: 2.23s
11:	learn: 0.6096172	total: 27.3ms	remaining: 2.25s
12:	learn: 0.6034787	total: 29.7ms	remaining: 2.26s
13:	learn: 0.5976496	total: 32.1ms	remaining: 2.26s
14:	learn: 0.5919432	total: 34.7ms	remaining: 2.27s
15:	learn: 0.5876777	total: 36.6ms	remaining: 2.25s
16:	learn: 0.5818113	total: 39.2ms	remaining: 2.27s
17:	learn: 0.5756954	total: 41.8ms	remaining: 2.28s
18:	learn: 0.5700415	total: 44.4ms	remaining: 2.29s
19:	learn: 0.5638244	to

228:	learn: 0.2069459	total: 474ms	remaining: 1.6s
229:	learn: 0.2062346	total: 476ms	remaining: 1.59s
230:	learn: 0.2056374	total: 478ms	remaining: 1.59s
231:	learn: 0.2047996	total: 480ms	remaining: 1.59s
232:	learn: 0.2042485	total: 482ms	remaining: 1.58s
233:	learn: 0.2037065	total: 483ms	remaining: 1.58s
234:	learn: 0.2034286	total: 485ms	remaining: 1.58s
235:	learn: 0.2030936	total: 487ms	remaining: 1.57s
236:	learn: 0.2024786	total: 489ms	remaining: 1.57s
237:	learn: 0.2019612	total: 491ms	remaining: 1.57s
238:	learn: 0.2016112	total: 493ms	remaining: 1.57s
239:	learn: 0.2008165	total: 495ms	remaining: 1.57s
240:	learn: 0.2001186	total: 497ms	remaining: 1.56s
241:	learn: 0.1996261	total: 499ms	remaining: 1.56s
242:	learn: 0.1988661	total: 501ms	remaining: 1.56s
243:	learn: 0.1983954	total: 503ms	remaining: 1.56s
244:	learn: 0.1976642	total: 505ms	remaining: 1.55s
245:	learn: 0.1971355	total: 507ms	remaining: 1.55s
246:	learn: 0.1964080	total: 509ms	remaining: 1.55s
247:	learn: 0

391:	learn: 0.1389486	total: 784ms	remaining: 1.22s
392:	learn: 0.1385527	total: 786ms	remaining: 1.21s
393:	learn: 0.1384139	total: 788ms	remaining: 1.21s
394:	learn: 0.1383095	total: 790ms	remaining: 1.21s
395:	learn: 0.1379397	total: 792ms	remaining: 1.21s
396:	learn: 0.1378748	total: 793ms	remaining: 1.2s
397:	learn: 0.1374406	total: 795ms	remaining: 1.2s
398:	learn: 0.1371090	total: 796ms	remaining: 1.2s
399:	learn: 0.1369025	total: 798ms	remaining: 1.2s
400:	learn: 0.1366081	total: 800ms	remaining: 1.2s
401:	learn: 0.1363014	total: 802ms	remaining: 1.19s
402:	learn: 0.1360266	total: 804ms	remaining: 1.19s
403:	learn: 0.1357965	total: 806ms	remaining: 1.19s
404:	learn: 0.1355425	total: 807ms	remaining: 1.19s
405:	learn: 0.1352453	total: 809ms	remaining: 1.18s
406:	learn: 0.1350253	total: 811ms	remaining: 1.18s
407:	learn: 0.1349434	total: 813ms	remaining: 1.18s
408:	learn: 0.1347039	total: 815ms	remaining: 1.18s
409:	learn: 0.1343357	total: 817ms	remaining: 1.18s
410:	learn: 0.134

627:	learn: 0.0855421	total: 1.26s	remaining: 744ms
628:	learn: 0.0855085	total: 1.26s	remaining: 742ms
629:	learn: 0.0854917	total: 1.26s	remaining: 740ms
630:	learn: 0.0853068	total: 1.26s	remaining: 738ms
631:	learn: 0.0852001	total: 1.26s	remaining: 736ms
632:	learn: 0.0849507	total: 1.26s	remaining: 733ms
633:	learn: 0.0847877	total: 1.27s	remaining: 731ms
634:	learn: 0.0846771	total: 1.27s	remaining: 729ms
635:	learn: 0.0844765	total: 1.27s	remaining: 727ms
636:	learn: 0.0842907	total: 1.27s	remaining: 725ms
637:	learn: 0.0841445	total: 1.27s	remaining: 723ms
638:	learn: 0.0840867	total: 1.27s	remaining: 721ms
639:	learn: 0.0839844	total: 1.28s	remaining: 719ms
640:	learn: 0.0837572	total: 1.28s	remaining: 717ms
641:	learn: 0.0836275	total: 1.28s	remaining: 715ms
642:	learn: 0.0833690	total: 1.28s	remaining: 713ms
643:	learn: 0.0830725	total: 1.29s	remaining: 711ms
644:	learn: 0.0829402	total: 1.29s	remaining: 709ms
645:	learn: 0.0826873	total: 1.29s	remaining: 707ms
646:	learn: 

790:	learn: 0.0661253	total: 1.57s	remaining: 415ms
791:	learn: 0.0661014	total: 1.57s	remaining: 413ms
792:	learn: 0.0659063	total: 1.57s	remaining: 411ms
793:	learn: 0.0657681	total: 1.58s	remaining: 409ms
794:	learn: 0.0656852	total: 1.58s	remaining: 407ms
795:	learn: 0.0655969	total: 1.58s	remaining: 405ms
796:	learn: 0.0655778	total: 1.58s	remaining: 403ms
797:	learn: 0.0655123	total: 1.58s	remaining: 401ms
798:	learn: 0.0655013	total: 1.58s	remaining: 399ms
799:	learn: 0.0654510	total: 1.59s	remaining: 397ms
800:	learn: 0.0654013	total: 1.59s	remaining: 395ms
801:	learn: 0.0652903	total: 1.59s	remaining: 393ms
802:	learn: 0.0652343	total: 1.59s	remaining: 391ms
803:	learn: 0.0650975	total: 1.59s	remaining: 389ms
804:	learn: 0.0650613	total: 1.6s	remaining: 387ms
805:	learn: 0.0649556	total: 1.6s	remaining: 385ms
806:	learn: 0.0648819	total: 1.6s	remaining: 383ms
807:	learn: 0.0648031	total: 1.6s	remaining: 381ms
808:	learn: 0.0647763	total: 1.6s	remaining: 379ms
809:	learn: 0.064

954:	learn: 0.0527651	total: 1.88s	remaining: 88.7ms
955:	learn: 0.0526615	total: 1.88s	remaining: 86.7ms
956:	learn: 0.0525443	total: 1.89s	remaining: 84.7ms
957:	learn: 0.0524623	total: 1.89s	remaining: 82.8ms
958:	learn: 0.0523668	total: 1.89s	remaining: 80.8ms
959:	learn: 0.0523495	total: 1.89s	remaining: 78.8ms
960:	learn: 0.0523410	total: 1.89s	remaining: 76.8ms
961:	learn: 0.0522427	total: 1.89s	remaining: 74.8ms
962:	learn: 0.0521953	total: 1.9s	remaining: 72.8ms
963:	learn: 0.0521797	total: 1.9s	remaining: 70.9ms
964:	learn: 0.0520807	total: 1.9s	remaining: 68.9ms
965:	learn: 0.0519437	total: 1.9s	remaining: 66.9ms
966:	learn: 0.0518463	total: 1.9s	remaining: 65ms
967:	learn: 0.0517321	total: 1.91s	remaining: 63ms
968:	learn: 0.0516462	total: 1.91s	remaining: 61ms
969:	learn: 0.0515746	total: 1.91s	remaining: 59.1ms
970:	learn: 0.0515671	total: 1.91s	remaining: 57.1ms
971:	learn: 0.0514394	total: 1.91s	remaining: 55.1ms
972:	learn: 0.0513450	total: 1.92s	remaining: 53.2ms
973:

In [69]:
grid_search_cat(params={'max_depth':[2, 3, 5, 6, 8]})

Learning rate set to 0.00547
0:	learn: 0.6904981	total: 1.44ms	remaining: 1.44s
1:	learn: 0.6858263	total: 2.6ms	remaining: 1.3s
2:	learn: 0.6842310	total: 3.5ms	remaining: 1.16s
3:	learn: 0.6817443	total: 4.39ms	remaining: 1.09s
4:	learn: 0.6782397	total: 6.07ms	remaining: 1.21s
5:	learn: 0.6756287	total: 7.72ms	remaining: 1.28s
6:	learn: 0.6725745	total: 8.74ms	remaining: 1.24s
7:	learn: 0.6697431	total: 9.72ms	remaining: 1.21s
8:	learn: 0.6673335	total: 10.8ms	remaining: 1.19s
9:	learn: 0.6642608	total: 11.8ms	remaining: 1.17s
10:	learn: 0.6617615	total: 12.7ms	remaining: 1.15s
11:	learn: 0.6591904	total: 13.8ms	remaining: 1.13s
12:	learn: 0.6563258	total: 14.8ms	remaining: 1.13s
13:	learn: 0.6526583	total: 15.8ms	remaining: 1.11s
14:	learn: 0.6498677	total: 16.9ms	remaining: 1.11s
15:	learn: 0.6466882	total: 17.9ms	remaining: 1.1s
16:	learn: 0.6430683	total: 19ms	remaining: 1.1s
17:	learn: 0.6398533	total: 20ms	remaining: 1.09s
18:	learn: 0.6371851	total: 20.9ms	remaining: 1.08s
19

284:	learn: 0.3629703	total: 283ms	remaining: 711ms
285:	learn: 0.3623990	total: 285ms	remaining: 711ms
286:	learn: 0.3619920	total: 286ms	remaining: 711ms
287:	learn: 0.3618129	total: 287ms	remaining: 710ms
288:	learn: 0.3615695	total: 288ms	remaining: 710ms
289:	learn: 0.3612073	total: 289ms	remaining: 709ms
290:	learn: 0.3607483	total: 290ms	remaining: 707ms
291:	learn: 0.3603110	total: 291ms	remaining: 706ms
292:	learn: 0.3599456	total: 292ms	remaining: 705ms
293:	learn: 0.3597031	total: 293ms	remaining: 704ms
294:	learn: 0.3595687	total: 294ms	remaining: 704ms
295:	learn: 0.3591150	total: 296ms	remaining: 703ms
296:	learn: 0.3588649	total: 296ms	remaining: 702ms
297:	learn: 0.3584322	total: 297ms	remaining: 701ms
298:	learn: 0.3579447	total: 299ms	remaining: 700ms
299:	learn: 0.3574188	total: 300ms	remaining: 699ms
300:	learn: 0.3570963	total: 301ms	remaining: 699ms
301:	learn: 0.3565673	total: 302ms	remaining: 698ms
302:	learn: 0.3563339	total: 303ms	remaining: 697ms
303:	learn: 

564:	learn: 0.2938453	total: 570ms	remaining: 439ms
565:	learn: 0.2937829	total: 572ms	remaining: 438ms
566:	learn: 0.2935847	total: 572ms	remaining: 437ms
567:	learn: 0.2933216	total: 574ms	remaining: 436ms
568:	learn: 0.2931594	total: 575ms	remaining: 435ms
569:	learn: 0.2930807	total: 576ms	remaining: 434ms
570:	learn: 0.2929833	total: 577ms	remaining: 433ms
571:	learn: 0.2928822	total: 578ms	remaining: 432ms
572:	learn: 0.2926949	total: 578ms	remaining: 431ms
573:	learn: 0.2924975	total: 579ms	remaining: 430ms
574:	learn: 0.2923823	total: 580ms	remaining: 429ms
575:	learn: 0.2922137	total: 581ms	remaining: 428ms
576:	learn: 0.2919205	total: 582ms	remaining: 427ms
577:	learn: 0.2917488	total: 584ms	remaining: 426ms
578:	learn: 0.2915002	total: 585ms	remaining: 425ms
579:	learn: 0.2913436	total: 586ms	remaining: 424ms
580:	learn: 0.2910954	total: 586ms	remaining: 423ms
581:	learn: 0.2908554	total: 588ms	remaining: 422ms
582:	learn: 0.2906186	total: 588ms	remaining: 421ms
583:	learn: 

853:	learn: 0.2532431	total: 855ms	remaining: 146ms
854:	learn: 0.2530997	total: 856ms	remaining: 145ms
855:	learn: 0.2529863	total: 857ms	remaining: 144ms
856:	learn: 0.2528966	total: 858ms	remaining: 143ms
857:	learn: 0.2527979	total: 859ms	remaining: 142ms
858:	learn: 0.2527341	total: 860ms	remaining: 141ms
859:	learn: 0.2525365	total: 861ms	remaining: 140ms
860:	learn: 0.2525174	total: 862ms	remaining: 139ms
861:	learn: 0.2524740	total: 863ms	remaining: 138ms
862:	learn: 0.2523589	total: 864ms	remaining: 137ms
863:	learn: 0.2522298	total: 865ms	remaining: 136ms
864:	learn: 0.2521427	total: 866ms	remaining: 135ms
865:	learn: 0.2520479	total: 867ms	remaining: 134ms
866:	learn: 0.2519362	total: 868ms	remaining: 133ms
867:	learn: 0.2517161	total: 869ms	remaining: 132ms
868:	learn: 0.2515864	total: 870ms	remaining: 131ms
869:	learn: 0.2514591	total: 871ms	remaining: 130ms
870:	learn: 0.2513291	total: 871ms	remaining: 129ms
871:	learn: 0.2512312	total: 872ms	remaining: 128ms
872:	learn: 

In [70]:
grid_search_cat(params={'min_child_samples':[1, 2, 3, 4, 5]})

Learning rate set to 0.00547
0:	learn: 0.6890586	total: 2.93ms	remaining: 2.93s
1:	learn: 0.6851369	total: 5.01ms	remaining: 2.5s
2:	learn: 0.6806205	total: 7.01ms	remaining: 2.33s
3:	learn: 0.6771860	total: 8.12ms	remaining: 2.02s
4:	learn: 0.6724788	total: 10.1ms	remaining: 2.01s
5:	learn: 0.6687116	total: 12.1ms	remaining: 2s
6:	learn: 0.6645896	total: 14.1ms	remaining: 2s
7:	learn: 0.6607750	total: 16.1ms	remaining: 1.99s
8:	learn: 0.6565425	total: 18.1ms	remaining: 2s
9:	learn: 0.6532435	total: 20.1ms	remaining: 1.99s
10:	learn: 0.6498891	total: 22.2ms	remaining: 2s
11:	learn: 0.6454730	total: 24.3ms	remaining: 2s
12:	learn: 0.6417368	total: 26.5ms	remaining: 2.01s
13:	learn: 0.6381601	total: 28.7ms	remaining: 2.02s
14:	learn: 0.6349286	total: 30.6ms	remaining: 2.01s
15:	learn: 0.6322543	total: 32.2ms	remaining: 1.98s
16:	learn: 0.6285952	total: 34.2ms	remaining: 1.98s
17:	learn: 0.6247744	total: 36.5ms	remaining: 1.99s
18:	learn: 0.6211876	total: 38.4ms	remaining: 1.98s
19:	learn

240:	learn: 0.2850592	total: 468ms	remaining: 1.47s
241:	learn: 0.2846623	total: 469ms	remaining: 1.47s
242:	learn: 0.2838244	total: 471ms	remaining: 1.47s
243:	learn: 0.2831586	total: 473ms	remaining: 1.47s
244:	learn: 0.2823715	total: 475ms	remaining: 1.46s
245:	learn: 0.2818948	total: 477ms	remaining: 1.46s
246:	learn: 0.2810808	total: 479ms	remaining: 1.46s
247:	learn: 0.2807604	total: 480ms	remaining: 1.46s
248:	learn: 0.2802589	total: 482ms	remaining: 1.45s
249:	learn: 0.2796876	total: 484ms	remaining: 1.45s
250:	learn: 0.2792346	total: 486ms	remaining: 1.45s
251:	learn: 0.2786385	total: 488ms	remaining: 1.45s
252:	learn: 0.2781027	total: 490ms	remaining: 1.45s
253:	learn: 0.2775620	total: 492ms	remaining: 1.44s
254:	learn: 0.2770335	total: 493ms	remaining: 1.44s
255:	learn: 0.2766422	total: 495ms	remaining: 1.44s
256:	learn: 0.2761788	total: 497ms	remaining: 1.44s
257:	learn: 0.2757570	total: 499ms	remaining: 1.43s
258:	learn: 0.2749305	total: 501ms	remaining: 1.43s
259:	learn: 

406:	learn: 0.2110591	total: 779ms	remaining: 1.13s
407:	learn: 0.2107472	total: 781ms	remaining: 1.13s
408:	learn: 0.2104486	total: 782ms	remaining: 1.13s
409:	learn: 0.2101670	total: 784ms	remaining: 1.13s
410:	learn: 0.2099380	total: 785ms	remaining: 1.13s
411:	learn: 0.2095882	total: 787ms	remaining: 1.12s
412:	learn: 0.2092611	total: 789ms	remaining: 1.12s
413:	learn: 0.2089104	total: 791ms	remaining: 1.12s
414:	learn: 0.2085915	total: 793ms	remaining: 1.12s
415:	learn: 0.2082909	total: 795ms	remaining: 1.11s
416:	learn: 0.2079204	total: 796ms	remaining: 1.11s
417:	learn: 0.2077011	total: 798ms	remaining: 1.11s
418:	learn: 0.2074844	total: 800ms	remaining: 1.11s
419:	learn: 0.2070312	total: 802ms	remaining: 1.11s
420:	learn: 0.2067244	total: 804ms	remaining: 1.1s
421:	learn: 0.2063663	total: 806ms	remaining: 1.1s
422:	learn: 0.2059107	total: 808ms	remaining: 1.1s
423:	learn: 0.2056360	total: 810ms	remaining: 1.1s
424:	learn: 0.2053749	total: 812ms	remaining: 1.1s
425:	learn: 0.204

569:	learn: 0.1661541	total: 1.09s	remaining: 823ms
570:	learn: 0.1659866	total: 1.09s	remaining: 821ms
571:	learn: 0.1657730	total: 1.09s	remaining: 819ms
572:	learn: 0.1655805	total: 1.09s	remaining: 817ms
573:	learn: 0.1653646	total: 1.1s	remaining: 815ms
574:	learn: 0.1651309	total: 1.1s	remaining: 813ms
575:	learn: 0.1648754	total: 1.1s	remaining: 810ms
576:	learn: 0.1646741	total: 1.1s	remaining: 809ms
577:	learn: 0.1644291	total: 1.1s	remaining: 807ms
578:	learn: 0.1642694	total: 1.11s	remaining: 805ms
579:	learn: 0.1639043	total: 1.11s	remaining: 803ms
580:	learn: 0.1637417	total: 1.11s	remaining: 801ms
581:	learn: 0.1634432	total: 1.11s	remaining: 799ms
582:	learn: 0.1632467	total: 1.11s	remaining: 798ms
583:	learn: 0.1630736	total: 1.12s	remaining: 796ms
584:	learn: 0.1628396	total: 1.12s	remaining: 794ms
585:	learn: 0.1626337	total: 1.12s	remaining: 792ms
586:	learn: 0.1624644	total: 1.12s	remaining: 790ms
587:	learn: 0.1620940	total: 1.12s	remaining: 788ms
588:	learn: 0.162

731:	learn: 0.1355499	total: 1.4s	remaining: 514ms
732:	learn: 0.1353891	total: 1.4s	remaining: 512ms
733:	learn: 0.1350518	total: 1.41s	remaining: 510ms
734:	learn: 0.1349428	total: 1.41s	remaining: 508ms
735:	learn: 0.1347434	total: 1.41s	remaining: 506ms
736:	learn: 0.1345369	total: 1.41s	remaining: 504ms
737:	learn: 0.1344337	total: 1.41s	remaining: 502ms
738:	learn: 0.1343430	total: 1.42s	remaining: 500ms
739:	learn: 0.1341344	total: 1.42s	remaining: 498ms
740:	learn: 0.1340570	total: 1.42s	remaining: 496ms
741:	learn: 0.1339212	total: 1.42s	remaining: 494ms
742:	learn: 0.1336508	total: 1.42s	remaining: 492ms
743:	learn: 0.1335370	total: 1.42s	remaining: 490ms
744:	learn: 0.1333388	total: 1.43s	remaining: 488ms
745:	learn: 0.1332058	total: 1.43s	remaining: 486ms
746:	learn: 0.1330226	total: 1.43s	remaining: 485ms
747:	learn: 0.1328009	total: 1.43s	remaining: 483ms
748:	learn: 0.1326031	total: 1.43s	remaining: 481ms
749:	learn: 0.1325151	total: 1.44s	remaining: 479ms
750:	learn: 0.

894:	learn: 0.1133855	total: 1.72s	remaining: 201ms
895:	learn: 0.1132700	total: 1.72s	remaining: 199ms
896:	learn: 0.1131045	total: 1.72s	remaining: 197ms
897:	learn: 0.1129440	total: 1.72s	remaining: 196ms
898:	learn: 0.1128621	total: 1.72s	remaining: 194ms
899:	learn: 0.1127573	total: 1.72s	remaining: 192ms
900:	learn: 0.1126583	total: 1.73s	remaining: 190ms
901:	learn: 0.1126487	total: 1.73s	remaining: 188ms
902:	learn: 0.1125373	total: 1.73s	remaining: 186ms
903:	learn: 0.1123018	total: 1.73s	remaining: 184ms
904:	learn: 0.1121455	total: 1.73s	remaining: 182ms
905:	learn: 0.1120003	total: 1.74s	remaining: 180ms
906:	learn: 0.1118675	total: 1.74s	remaining: 178ms
907:	learn: 0.1116954	total: 1.74s	remaining: 176ms
908:	learn: 0.1115919	total: 1.74s	remaining: 174ms
909:	learn: 0.1114582	total: 1.74s	remaining: 172ms
910:	learn: 0.1113787	total: 1.74s	remaining: 170ms
911:	learn: 0.1112342	total: 1.75s	remaining: 169ms
912:	learn: 0.1111286	total: 1.75s	remaining: 167ms
913:	learn: 

In [71]:
grid_search_cat(params={'subsample':[0.5, 0.7, 0.8, 0.9, 1]})

Learning rate set to 0.00547
0:	learn: 0.6900126	total: 2.44ms	remaining: 2.43s
1:	learn: 0.6862175	total: 4.36ms	remaining: 2.18s
2:	learn: 0.6815702	total: 6.28ms	remaining: 2.09s
3:	learn: 0.6781255	total: 7.35ms	remaining: 1.83s
4:	learn: 0.6755104	total: 9.21ms	remaining: 1.83s
5:	learn: 0.6720810	total: 11.4ms	remaining: 1.89s
6:	learn: 0.6678784	total: 13.5ms	remaining: 1.92s
7:	learn: 0.6634892	total: 15.4ms	remaining: 1.91s
8:	learn: 0.6591589	total: 17.3ms	remaining: 1.9s
9:	learn: 0.6566933	total: 19.3ms	remaining: 1.91s
10:	learn: 0.6544694	total: 21.2ms	remaining: 1.91s
11:	learn: 0.6500247	total: 23.2ms	remaining: 1.91s
12:	learn: 0.6464511	total: 25.2ms	remaining: 1.91s
13:	learn: 0.6429841	total: 27.1ms	remaining: 1.91s
14:	learn: 0.6396316	total: 29.2ms	remaining: 1.91s
15:	learn: 0.6366075	total: 30.6ms	remaining: 1.88s
16:	learn: 0.6333351	total: 32.6ms	remaining: 1.89s
17:	learn: 0.6302939	total: 34.7ms	remaining: 1.89s
18:	learn: 0.6268328	total: 36.6ms	remaining: 

240:	learn: 0.2957415	total: 464ms	remaining: 1.46s
241:	learn: 0.2951221	total: 465ms	remaining: 1.46s
242:	learn: 0.2943978	total: 467ms	remaining: 1.46s
243:	learn: 0.2937143	total: 469ms	remaining: 1.45s
244:	learn: 0.2931383	total: 471ms	remaining: 1.45s
245:	learn: 0.2924537	total: 472ms	remaining: 1.45s
246:	learn: 0.2918338	total: 474ms	remaining: 1.45s
247:	learn: 0.2911394	total: 476ms	remaining: 1.44s
248:	learn: 0.2906130	total: 478ms	remaining: 1.44s
249:	learn: 0.2902863	total: 479ms	remaining: 1.44s
250:	learn: 0.2894537	total: 482ms	remaining: 1.44s
251:	learn: 0.2885893	total: 484ms	remaining: 1.44s
252:	learn: 0.2879938	total: 485ms	remaining: 1.43s
253:	learn: 0.2873852	total: 488ms	remaining: 1.43s
254:	learn: 0.2871034	total: 489ms	remaining: 1.43s
255:	learn: 0.2862446	total: 491ms	remaining: 1.43s
256:	learn: 0.2856244	total: 493ms	remaining: 1.43s
257:	learn: 0.2851930	total: 495ms	remaining: 1.42s
258:	learn: 0.2847500	total: 497ms	remaining: 1.42s
259:	learn: 

404:	learn: 0.2228100	total: 776ms	remaining: 1.14s
405:	learn: 0.2226197	total: 778ms	remaining: 1.14s
406:	learn: 0.2222811	total: 780ms	remaining: 1.14s
407:	learn: 0.2217969	total: 782ms	remaining: 1.13s
408:	learn: 0.2213019	total: 784ms	remaining: 1.13s
409:	learn: 0.2210029	total: 785ms	remaining: 1.13s
410:	learn: 0.2205789	total: 788ms	remaining: 1.13s
411:	learn: 0.2200031	total: 790ms	remaining: 1.13s
412:	learn: 0.2194407	total: 792ms	remaining: 1.13s
413:	learn: 0.2191046	total: 794ms	remaining: 1.12s
414:	learn: 0.2186694	total: 796ms	remaining: 1.12s
415:	learn: 0.2184309	total: 797ms	remaining: 1.12s
416:	learn: 0.2181170	total: 800ms	remaining: 1.12s
417:	learn: 0.2178352	total: 802ms	remaining: 1.12s
418:	learn: 0.2175514	total: 804ms	remaining: 1.11s
419:	learn: 0.2170590	total: 806ms	remaining: 1.11s
420:	learn: 0.2167422	total: 807ms	remaining: 1.11s
421:	learn: 0.2162926	total: 809ms	remaining: 1.11s
422:	learn: 0.2159859	total: 811ms	remaining: 1.11s
423:	learn: 

565:	learn: 0.1779322	total: 1.09s	remaining: 833ms
566:	learn: 0.1778411	total: 1.09s	remaining: 831ms
567:	learn: 0.1776608	total: 1.09s	remaining: 829ms
568:	learn: 0.1775235	total: 1.09s	remaining: 827ms
569:	learn: 0.1772563	total: 1.09s	remaining: 825ms
570:	learn: 0.1770587	total: 1.09s	remaining: 823ms
571:	learn: 0.1767861	total: 1.1s	remaining: 821ms
572:	learn: 0.1765295	total: 1.1s	remaining: 820ms
573:	learn: 0.1763193	total: 1.1s	remaining: 818ms
574:	learn: 0.1759011	total: 1.1s	remaining: 816ms
575:	learn: 0.1757333	total: 1.1s	remaining: 814ms
576:	learn: 0.1753978	total: 1.11s	remaining: 812ms
577:	learn: 0.1751896	total: 1.11s	remaining: 810ms
578:	learn: 0.1748987	total: 1.11s	remaining: 808ms
579:	learn: 0.1747027	total: 1.11s	remaining: 807ms
580:	learn: 0.1744386	total: 1.11s	remaining: 805ms
581:	learn: 0.1739992	total: 1.12s	remaining: 803ms
582:	learn: 0.1737620	total: 1.12s	remaining: 801ms
583:	learn: 0.1736144	total: 1.12s	remaining: 799ms
584:	learn: 0.173

725:	learn: 0.1457653	total: 1.4s	remaining: 527ms
726:	learn: 0.1456849	total: 1.4s	remaining: 525ms
727:	learn: 0.1455665	total: 1.4s	remaining: 523ms
728:	learn: 0.1454876	total: 1.4s	remaining: 521ms
729:	learn: 0.1452646	total: 1.4s	remaining: 519ms
730:	learn: 0.1450638	total: 1.4s	remaining: 517ms
731:	learn: 0.1448558	total: 1.41s	remaining: 515ms
732:	learn: 0.1446383	total: 1.41s	remaining: 513ms
733:	learn: 0.1445708	total: 1.41s	remaining: 511ms
734:	learn: 0.1443984	total: 1.41s	remaining: 509ms
735:	learn: 0.1441394	total: 1.41s	remaining: 507ms
736:	learn: 0.1439176	total: 1.42s	remaining: 505ms
737:	learn: 0.1437226	total: 1.42s	remaining: 503ms
738:	learn: 0.1436015	total: 1.42s	remaining: 501ms
739:	learn: 0.1435011	total: 1.42s	remaining: 499ms
740:	learn: 0.1433678	total: 1.42s	remaining: 498ms
741:	learn: 0.1431492	total: 1.43s	remaining: 496ms
742:	learn: 0.1431104	total: 1.43s	remaining: 494ms
743:	learn: 0.1429033	total: 1.43s	remaining: 492ms
744:	learn: 0.1428

889:	learn: 0.1221852	total: 1.71s	remaining: 211ms
890:	learn: 0.1220267	total: 1.71s	remaining: 209ms
891:	learn: 0.1218345	total: 1.71s	remaining: 207ms
892:	learn: 0.1216602	total: 1.71s	remaining: 205ms
893:	learn: 0.1214977	total: 1.72s	remaining: 203ms
894:	learn: 0.1213641	total: 1.72s	remaining: 201ms
895:	learn: 0.1211689	total: 1.72s	remaining: 199ms
896:	learn: 0.1209391	total: 1.72s	remaining: 198ms
897:	learn: 0.1207880	total: 1.72s	remaining: 196ms
898:	learn: 0.1205989	total: 1.72s	remaining: 194ms
899:	learn: 0.1204159	total: 1.73s	remaining: 192ms
900:	learn: 0.1202898	total: 1.73s	remaining: 190ms
901:	learn: 0.1201796	total: 1.73s	remaining: 188ms
902:	learn: 0.1199650	total: 1.73s	remaining: 186ms
903:	learn: 0.1198195	total: 1.73s	remaining: 184ms
904:	learn: 0.1197013	total: 1.74s	remaining: 182ms
905:	learn: 0.1195593	total: 1.74s	remaining: 180ms
906:	learn: 0.1194555	total: 1.74s	remaining: 178ms
907:	learn: 0.1193447	total: 1.74s	remaining: 176ms
908:	learn: 

In [72]:
grid_search_cat(params={'colsample_bylevel':[0.5, 0.7, 0.8, 0.9, 1]})

Learning rate set to 0.00547
0:	learn: 0.6899059	total: 2.14ms	remaining: 2.14s
1:	learn: 0.6867030	total: 3.82ms	remaining: 1.9s
2:	learn: 0.6822403	total: 5.86ms	remaining: 1.95s
3:	learn: 0.6780679	total: 8.09ms	remaining: 2.02s
4:	learn: 0.6736924	total: 10.4ms	remaining: 2.07s
5:	learn: 0.6692816	total: 12.6ms	remaining: 2.08s
6:	learn: 0.6645121	total: 14.9ms	remaining: 2.12s
7:	learn: 0.6603376	total: 17.2ms	remaining: 2.13s
8:	learn: 0.6564364	total: 19.5ms	remaining: 2.15s
9:	learn: 0.6527910	total: 21.5ms	remaining: 2.13s
10:	learn: 0.6488161	total: 23.6ms	remaining: 2.13s
11:	learn: 0.6450743	total: 26ms	remaining: 2.14s
12:	learn: 0.6423637	total: 27.9ms	remaining: 2.12s
13:	learn: 0.6383658	total: 29.9ms	remaining: 2.11s
14:	learn: 0.6349154	total: 32.3ms	remaining: 2.12s
15:	learn: 0.6311591	total: 34.2ms	remaining: 2.11s
16:	learn: 0.6273384	total: 36.4ms	remaining: 2.11s
17:	learn: 0.6236749	total: 38.8ms	remaining: 2.12s
18:	learn: 0.6205024	total: 41.3ms	remaining: 2.

220:	learn: 0.3018501	total: 471ms	remaining: 1.66s
221:	learn: 0.3011038	total: 474ms	remaining: 1.66s
222:	learn: 0.3003725	total: 476ms	remaining: 1.66s
223:	learn: 0.2994113	total: 478ms	remaining: 1.65s
224:	learn: 0.2988074	total: 479ms	remaining: 1.65s
225:	learn: 0.2981566	total: 482ms	remaining: 1.65s
226:	learn: 0.2974035	total: 483ms	remaining: 1.65s
227:	learn: 0.2968654	total: 497ms	remaining: 1.68s
228:	learn: 0.2965357	total: 498ms	remaining: 1.68s
229:	learn: 0.2960195	total: 500ms	remaining: 1.67s
230:	learn: 0.2954864	total: 502ms	remaining: 1.67s
231:	learn: 0.2945146	total: 504ms	remaining: 1.67s
232:	learn: 0.2940257	total: 507ms	remaining: 1.67s
233:	learn: 0.2932153	total: 509ms	remaining: 1.67s
234:	learn: 0.2929725	total: 511ms	remaining: 1.66s
235:	learn: 0.2924268	total: 513ms	remaining: 1.66s
236:	learn: 0.2920098	total: 515ms	remaining: 1.66s
237:	learn: 0.2915182	total: 517ms	remaining: 1.66s
238:	learn: 0.2908575	total: 520ms	remaining: 1.65s
239:	learn: 

422:	learn: 0.2073077	total: 953ms	remaining: 1.3s
423:	learn: 0.2069078	total: 954ms	remaining: 1.3s
424:	learn: 0.2066025	total: 957ms	remaining: 1.29s
425:	learn: 0.2064314	total: 959ms	remaining: 1.29s
426:	learn: 0.2060238	total: 961ms	remaining: 1.29s
427:	learn: 0.2057035	total: 963ms	remaining: 1.29s
428:	learn: 0.2055912	total: 965ms	remaining: 1.28s
429:	learn: 0.2053173	total: 967ms	remaining: 1.28s
430:	learn: 0.2050177	total: 969ms	remaining: 1.28s
431:	learn: 0.2046230	total: 971ms	remaining: 1.28s
432:	learn: 0.2044450	total: 973ms	remaining: 1.27s
433:	learn: 0.2042723	total: 975ms	remaining: 1.27s
434:	learn: 0.2038948	total: 978ms	remaining: 1.27s
435:	learn: 0.2034496	total: 979ms	remaining: 1.27s
436:	learn: 0.2031398	total: 981ms	remaining: 1.26s
437:	learn: 0.2027091	total: 984ms	remaining: 1.26s
438:	learn: 0.2022645	total: 986ms	remaining: 1.26s
439:	learn: 0.2020954	total: 988ms	remaining: 1.26s
440:	learn: 0.2017988	total: 990ms	remaining: 1.25s
441:	learn: 0.

638:	learn: 0.1552449	total: 1.42s	remaining: 804ms
639:	learn: 0.1550113	total: 1.43s	remaining: 802ms
640:	learn: 0.1548139	total: 1.43s	remaining: 800ms
641:	learn: 0.1546348	total: 1.43s	remaining: 797ms
642:	learn: 0.1544220	total: 1.43s	remaining: 795ms
643:	learn: 0.1541586	total: 1.43s	remaining: 792ms
644:	learn: 0.1539946	total: 1.44s	remaining: 790ms
645:	learn: 0.1537376	total: 1.44s	remaining: 788ms
646:	learn: 0.1534976	total: 1.44s	remaining: 785ms
647:	learn: 0.1533579	total: 1.44s	remaining: 783ms
648:	learn: 0.1532048	total: 1.44s	remaining: 781ms
649:	learn: 0.1531874	total: 1.45s	remaining: 779ms
650:	learn: 0.1529677	total: 1.45s	remaining: 776ms
651:	learn: 0.1528179	total: 1.45s	remaining: 774ms
652:	learn: 0.1526197	total: 1.45s	remaining: 772ms
653:	learn: 0.1524811	total: 1.46s	remaining: 770ms
654:	learn: 0.1523126	total: 1.46s	remaining: 768ms
655:	learn: 0.1520626	total: 1.46s	remaining: 765ms
656:	learn: 0.1517602	total: 1.46s	remaining: 763ms
657:	learn: 

847:	learn: 0.1222208	total: 1.9s	remaining: 340ms
848:	learn: 0.1219874	total: 1.9s	remaining: 337ms
849:	learn: 0.1218296	total: 1.9s	remaining: 335ms
850:	learn: 0.1216078	total: 1.9s	remaining: 333ms
851:	learn: 0.1214015	total: 1.9s	remaining: 331ms
852:	learn: 0.1212696	total: 1.91s	remaining: 328ms
853:	learn: 0.1211857	total: 1.91s	remaining: 326ms
854:	learn: 0.1210680	total: 1.91s	remaining: 324ms
855:	learn: 0.1210123	total: 1.91s	remaining: 322ms
856:	learn: 0.1208876	total: 1.91s	remaining: 319ms
857:	learn: 0.1207491	total: 1.92s	remaining: 317ms
858:	learn: 0.1205758	total: 1.92s	remaining: 315ms
859:	learn: 0.1205177	total: 1.92s	remaining: 313ms
860:	learn: 0.1204573	total: 1.92s	remaining: 311ms
861:	learn: 0.1202809	total: 1.93s	remaining: 308ms
862:	learn: 0.1201065	total: 1.93s	remaining: 306ms
863:	learn: 0.1199329	total: 1.93s	remaining: 304ms
864:	learn: 0.1198471	total: 1.93s	remaining: 302ms
865:	learn: 0.1196737	total: 1.94s	remaining: 299ms
866:	learn: 0.119

In [74]:
grid_search_cat(params={'n_estimators':[700,1000],'learning_rate':[0.01,0.02],
                      'max_depth':[3], 
                      'subsample':[0.45,0.5,0.6],'colsample_bylevel':[0.7,0.8,0.9]})

0:	learn: 0.6893710	total: 1.49ms	remaining: 1.04s
1:	learn: 0.6843263	total: 3.1ms	remaining: 1.08s
2:	learn: 0.6793010	total: 4.45ms	remaining: 1.03s
3:	learn: 0.6740970	total: 5.51ms	remaining: 959ms
4:	learn: 0.6692240	total: 6.45ms	remaining: 897ms
5:	learn: 0.6651337	total: 7.4ms	remaining: 856ms
6:	learn: 0.6605538	total: 8.36ms	remaining: 828ms
7:	learn: 0.6547907	total: 9.31ms	remaining: 805ms
8:	learn: 0.6499212	total: 10.3ms	remaining: 787ms
9:	learn: 0.6462709	total: 11.2ms	remaining: 772ms
10:	learn: 0.6430077	total: 12.1ms	remaining: 758ms
11:	learn: 0.6378563	total: 13.1ms	remaining: 749ms
12:	learn: 0.6350659	total: 14ms	remaining: 740ms
13:	learn: 0.6302020	total: 15ms	remaining: 733ms
14:	learn: 0.6251925	total: 15.9ms	remaining: 725ms
15:	learn: 0.6208691	total: 16.8ms	remaining: 717ms
16:	learn: 0.6170217	total: 17.7ms	remaining: 713ms
17:	learn: 0.6152501	total: 18.7ms	remaining: 707ms
18:	learn: 0.6119031	total: 19.6ms	remaining: 703ms
19:	learn: 0.6065910	total: 

278:	learn: 0.3144498	total: 278ms	remaining: 419ms
279:	learn: 0.3141869	total: 279ms	remaining: 419ms
280:	learn: 0.3136741	total: 281ms	remaining: 419ms
281:	learn: 0.3132698	total: 282ms	remaining: 418ms
282:	learn: 0.3128013	total: 283ms	remaining: 417ms
283:	learn: 0.3122929	total: 284ms	remaining: 416ms
284:	learn: 0.3119346	total: 285ms	remaining: 415ms
285:	learn: 0.3115855	total: 286ms	remaining: 413ms
286:	learn: 0.3113461	total: 287ms	remaining: 412ms
287:	learn: 0.3111657	total: 288ms	remaining: 412ms
288:	learn: 0.3108762	total: 289ms	remaining: 411ms
289:	learn: 0.3102480	total: 290ms	remaining: 409ms
290:	learn: 0.3101482	total: 290ms	remaining: 408ms
291:	learn: 0.3098534	total: 291ms	remaining: 407ms
292:	learn: 0.3096911	total: 293ms	remaining: 406ms
293:	learn: 0.3093712	total: 294ms	remaining: 405ms
294:	learn: 0.3090739	total: 295ms	remaining: 404ms
295:	learn: 0.3086193	total: 296ms	remaining: 404ms
296:	learn: 0.3084846	total: 297ms	remaining: 403ms
297:	learn: 

563:	learn: 0.2460722	total: 564ms	remaining: 136ms
564:	learn: 0.2459678	total: 565ms	remaining: 135ms
565:	learn: 0.2457999	total: 566ms	remaining: 134ms
566:	learn: 0.2456955	total: 567ms	remaining: 133ms
567:	learn: 0.2455192	total: 568ms	remaining: 132ms
568:	learn: 0.2454432	total: 569ms	remaining: 131ms
569:	learn: 0.2453445	total: 570ms	remaining: 130ms
570:	learn: 0.2452835	total: 571ms	remaining: 129ms
571:	learn: 0.2449461	total: 572ms	remaining: 128ms
572:	learn: 0.2446208	total: 572ms	remaining: 127ms
573:	learn: 0.2444722	total: 573ms	remaining: 126ms
574:	learn: 0.2442236	total: 574ms	remaining: 125ms
575:	learn: 0.2440770	total: 575ms	remaining: 124ms
576:	learn: 0.2439306	total: 577ms	remaining: 123ms
577:	learn: 0.2437141	total: 578ms	remaining: 122ms
578:	learn: 0.2436449	total: 579ms	remaining: 121ms
579:	learn: 0.2434324	total: 580ms	remaining: 120ms
580:	learn: 0.2433863	total: 581ms	remaining: 119ms
581:	learn: 0.2433165	total: 582ms	remaining: 118ms
582:	learn: 

In [75]:
model =CatBoostClassifier(max_depth=3,
                          subsample=0.5,
                          colsample_bylevel=0.7,
                          learning_rate=0.01, 
                          random_state=2, 
                          n_estimators=5000)
eval_set = [(X_val, y_val)]
model.fit(X_train_p, y_train_p,early_stopping_rounds=10, eval_set=eval_set)
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

0:	learn: 0.6885877	test: 0.6901639	best: 0.6901639 (0)	total: 1.25ms	remaining: 6.23s
1:	learn: 0.6839160	test: 0.6866463	best: 0.6866463 (1)	total: 2.32ms	remaining: 5.8s
2:	learn: 0.6766229	test: 0.6826979	best: 0.6826979 (2)	total: 3.29ms	remaining: 5.49s
3:	learn: 0.6705860	test: 0.6793726	best: 0.6793726 (3)	total: 4.3ms	remaining: 5.37s
4:	learn: 0.6657474	test: 0.6762903	best: 0.6762903 (4)	total: 5.32ms	remaining: 5.31s
5:	learn: 0.6595677	test: 0.6731836	best: 0.6731836 (5)	total: 6.29ms	remaining: 5.24s
6:	learn: 0.6555807	test: 0.6686029	best: 0.6686029 (6)	total: 7.41ms	remaining: 5.29s
7:	learn: 0.6505169	test: 0.6646456	best: 0.6646456 (7)	total: 8.41ms	remaining: 5.25s
8:	learn: 0.6467334	test: 0.6616286	best: 0.6616286 (8)	total: 9.33ms	remaining: 5.17s
9:	learn: 0.6433569	test: 0.6590299	best: 0.6590299 (9)	total: 10.4ms	remaining: 5.18s
10:	learn: 0.6388522	test: 0.6541024	best: 0.6541024 (10)	total: 11.4ms	remaining: 5.15s
11:	learn: 0.6342384	test: 0.6505888	best: 

138:	learn: 0.3741309	test: 0.4687540	best: 0.4687540 (138)	total: 140ms	remaining: 4.88s
139:	learn: 0.3736927	test: 0.4689093	best: 0.4687540 (138)	total: 141ms	remaining: 4.89s
140:	learn: 0.3723612	test: 0.4672977	best: 0.4672977 (140)	total: 142ms	remaining: 4.9s
141:	learn: 0.3715698	test: 0.4675043	best: 0.4672977 (140)	total: 143ms	remaining: 4.89s
142:	learn: 0.3709081	test: 0.4666152	best: 0.4666152 (142)	total: 144ms	remaining: 4.88s
143:	learn: 0.3700016	test: 0.4650704	best: 0.4650704 (143)	total: 145ms	remaining: 4.88s
144:	learn: 0.3693131	test: 0.4644664	best: 0.4644664 (144)	total: 146ms	remaining: 4.88s
145:	learn: 0.3685153	test: 0.4635203	best: 0.4635203 (145)	total: 147ms	remaining: 4.88s
146:	learn: 0.3679978	test: 0.4636799	best: 0.4635203 (145)	total: 148ms	remaining: 4.88s
147:	learn: 0.3672220	test: 0.4633054	best: 0.4633054 (147)	total: 149ms	remaining: 4.88s
148:	learn: 0.3661954	test: 0.4625186	best: 0.4625186 (148)	total: 150ms	remaining: 4.88s
149:	learn:

In [78]:
model_catc = CatBoostClassifier(n_estimators=183,
                                max_depth=3,
                                subsample=0.5,
                                colsample_bylevel=0.7,
                                learning_rate=0.01,
                                random_state=2)

model_catc.fit(X_train, y_train)
y_tpred=model_catc.predict(X_train)

t_accuracy=accuracy_score(y_train,y_tpred)
print('Training accuracy',t_accuracy)

# predictions for test data
y_pred = model_catc.predict(X_test)

# evaluate predictions
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

0:	learn: 0.6905737	total: 1ms	remaining: 183ms
1:	learn: 0.6878527	total: 1.69ms	remaining: 153ms
2:	learn: 0.6851999	total: 2.41ms	remaining: 145ms
3:	learn: 0.6819988	total: 3.37ms	remaining: 151ms
4:	learn: 0.6789122	total: 4.14ms	remaining: 147ms
5:	learn: 0.6758931	total: 4.99ms	remaining: 147ms
6:	learn: 0.6725318	total: 5.6ms	remaining: 141ms
7:	learn: 0.6703578	total: 6.25ms	remaining: 137ms
8:	learn: 0.6676753	total: 6.89ms	remaining: 133ms
9:	learn: 0.6650787	total: 7.45ms	remaining: 129ms
10:	learn: 0.6631943	total: 8.05ms	remaining: 126ms
11:	learn: 0.6603173	total: 9.05ms	remaining: 129ms
12:	learn: 0.6584024	total: 9.72ms	remaining: 127ms
13:	learn: 0.6553005	total: 10.4ms	remaining: 125ms
14:	learn: 0.6526293	total: 11ms	remaining: 123ms
15:	learn: 0.6497953	total: 11.9ms	remaining: 124ms
16:	learn: 0.6473618	total: 12.5ms	remaining: 122ms
17:	learn: 0.6445603	total: 13.2ms	remaining: 121ms
18:	learn: 0.6415757	total: 13.9ms	remaining: 120ms
19:	learn: 0.6389256	total: 

Training accuracy 0.8678414096916299
Accuracy: 85.53%
